<a href="https://colab.research.google.com/github/muratcankoylan/MuratcanKoylan-MarketingStructuralPrompts/blob/main/command_r_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install youtube-transcript-api

In [ ]:
!pip install anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.1/869.1 kB 5.7 MB/s eta 0:00:00


In [ ]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.8 MB/s eta 0:00:00


In [ ]:
pip install cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.3/144.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving cnbcmakeit1.csv to cnbcmakeit1.csv


In [ ]:
!ls


 cnbcmakeit1.csv       dragonsdenuk2.csv        garytan2.csv   garytanexports2.csv   sample_data
 dragonsdencont2.csv  'dragonsdenuk4 (1).csv'   garytan5.csv   garytanexports5.csv
 dragonsdencont3.csv   dragonsdenuk4.csv        garytan6.csv   garytanexports6.csv
 dragonsdencont5.csv   dragonsdenuk5.csv        garytan7.csv   garytanexports7.csv
 dragonsdencont.csv    dragonsdenuk.csv         garytan.csv    garytanexports.csv


In [ ]:
import pandas as pd

file_path = 'cnbcmakeit1.csv'
data = pd.read_csv(file_path)
print(data.head())  # This will display the first few rows of the DataFrame.

   https://www.youtube.com/watch?v=x7pGM5FSTpI
0  https://www.youtube.com/watch?v=LbWVysxsOhQ
1  https://www.youtube.com/watch?v=zUN9KQpUgzM
2  https://www.youtube.com/watch?v=PtLjacVrdFc
3  https://www.youtube.com/watch?v=7xbp7TYk7SU
4  https://www.youtube.com/watch?v=pG_xSaaFONY


In [67]:
import requests
import re
from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi
import json
import csv
import cohere

COHERE_API_KEY = "iBLmU6EZ9rXbK94jC3QhUqrmFTem9IoijEuUFxJR"

def extract_video_id(url):
    video_id = None
    if 'youtu.be' in url:
        video_id = url.split('/')[-1]
    elif 'youtube.com' in url:
        match = re.search(r'v=([^&]+)', url)
        if match:
            video_id = match.group(1)
    return video_id

def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        transcript_text = ' '.join([entry['text'] for entry in transcript])
        return transcript_text
    except Exception as e:
        print(f"Error retrieving transcript for video {video_id}: {str(e)}")
        return None

def generate_text(prompt, model="command-r", max_tokens=4000, temperature=0.3):
    headers = {
        "Authorization": f"Bearer {COHERE_API_KEY}",
        "Content-Type": "application/json"
    }
    data = {
        "message": prompt,
        "model": model,
        "max_tokens": max_tokens,
        "temperature": temperature,
    }
    response = requests.post("https://api.cohere.ai/v1/chat", headers=headers, json=data)
    if response.status_code == 200:
        try:
            response_data = response.json()
            if 'text' in response_data:
                return response_data['text']
            else:
                print("Unexpected response structure:", response_data)
                return None
        except json.JSONDecodeError as e:
            print("Failed to decode JSON from response:", str(e))
            return None
    else:
        print("Failed API call. Status code:", response.status_code, "Response body:", response.text)
        return None

def get_video_details(video_url):
    try:
        video = YouTube(video_url)
        video_length = video.length
        return {
            "title": video.title,
            "url": video_url,
            "description": video.description,
            "length": video_length
        }
    except Exception as e:
        print(f"Error retrieving video details for {video_url}: {str(e)}")
        return None

def generate_structured_business_ideas(transcript, video_details):
    predefined_categories = [
        "Subscription Services", "Software and Tech", "Social Media and Content Creation",
        "Retail and E-Commerce", "Health and Wellness", "Education and E-Learning",
        "Financial Services", "Sustainable and Eco-Friendly", "Food and Beverage",
        "Entertainment and Leisure", "Artificial Intelligence (AI)", "Media & Marketing"
    ]

    prompt = (
        f"Based on the transcript from '{video_details['title']}', generate several unique business ideas...\n\n"
        "Business Idea Name: {{name}} [Not the brand name! What is the business idea, write it. For example: Virtual Classroom Platform]\n"
        "Summary of the Idea: {{summary}} [150-200 characters. The format should be descriptive. ALways start with A or AN, like (An online platform that allows customers to design and order custom-made, eco-friendly home decor items, such as wall art, throw pillows, and rugs, using sustainable materials.)]\n"
        "Potential Roadmap: {{roadmap}} [Create a detailed business plan to launch this idea in 150-200 characters. The format should be descriptive. Example: (Identify and partner with ethical, sustainable brands and artisans to build a diverse product selection. Leverage social media and content marketing to educate and engage your target audience on the benefits of sustainable living. Offer personalized shopping experiences and exceptional customer service to build a loyal customer base.)]\n"
        "Category 1: {{category1}} [Choose the most relevant one, only one: B2B, B2C, B2G, C2C, D2C]\n"
        "Category 2: {{category2}} [Choose the most relevant one, only one: Very Easy, Easy, Moderate, Hard, Extreme]\n"
        "Category 3: {{category3}} [Choose the most relevant one, only one from the predefined list: "
        + ", ".join(predefined_categories) + "]\n"
        "Category 4: {{category4}} [Expected budget range: Be encouraging with your pricing and try to give the budget less than expected, use us dollar sign. Write two number with a 5, 10 or 50k differences. For example: $ 15.000 - 20.000]\n"
        f"{transcript}"
    )
    structured_ideas = generate_text(prompt)
    return structured_ideas


def parse_ideas_to_json(ideas_text, video_details, output_as_list=False):
    lines = ideas_text.split('\n')
    ideas = []
    idea = {}
    # Include 'Video Title' in your headers list
    headers = ['Business Idea Name', 'Summary of the Idea', 'Potential Roadmap', 'Category 1', 'Category 2', 'Category 3', 'Category 4', 'Video URL', 'Video Length', 'Video Title']

    for line in lines:
        if any(header in line for header in headers):
            key, value = line.split(': ', 1)
            key = key.strip()
            value = value.strip()
            if key == 'Business Idea Name' and idea:
                # Add video details to the idea before starting a new one
                idea['Video URL'] = video_details['url']
                idea['Video Length'] = video_details['length']
                idea['Video Title'] = video_details['title']  # Add the video title here
                ideas.append(idea)
                idea = {}
            idea[key] = value
    if idea:
        # Add video details to the last idea
        idea['Video URL'] = video_details['url']
        idea['Video Length'] = video_details['length']
        idea['Video Title'] = video_details['title']
        ideas.append(idea)

    if output_as_list:
        return ideas
    else:
        return json.dumps(ideas, indent=4)
def main():
    print("Script started")
    # Define the path to your input CSV file
    csv_input_path = 'cnbcmakeit1.csv'  # Make sure this matches your uploaded file name

    # Open the CSV file and read the video URLs
    video_urls = []  # We will store our URLs in this list
    with open(csv_input_path, mode='r', newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:
            if row:  # Check if row is not empty
                video_urls.append(row[0])  # Assume URL is in the first column of each row


    # Define the headers for the CSV where you will write the business ideas
    headers = ['Business Idea Name', 'Summary of the Idea', 'Potential Roadmap', 'Category 1', 'Category 2', 'Category 3', 'Category 4', 'Video URL', 'Video Length', 'Video Title']

    # Define the path to your output CSV file
    csv_output_path = 'cnbcmakeitexports1.csv'

    # Open the CSV file to append the ideas
    with open(csv_output_path, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        # Write the headers only if the file is new or empty
        if file.tell() == 0:
            writer.writerow(headers)

        # Process each video URL
        for link in video_urls:
            print(f"Processing video link: {link}")
            video_id = extract_video_id(link)
            if video_id:
                video_details = get_video_details(link)
                if video_details:
                    print(f"Video Title: {video_details['title']}")
                    transcript = get_transcript(video_id)
                    if transcript:
                        business_ideas_text = generate_structured_business_ideas(transcript, video_details)
                        ideas = parse_ideas_to_json(business_ideas_text, video_details, output_as_list=True)
                        for idea in ideas:
                            row = [idea.get(header, '') for header in headers]
                            writer.writerow(row)
                        print(f"Business ideas written for video {video_id}:")
                        print("---")
                    else:
                        print("No transcript available for this video.")

if __name__ == "__main__":
    main()

Script started
Processing video link: https://www.youtube.com/watch?v=x7pGM5FSTpI
Video Title: I Started A Business With $1,000 - Now It Brings In Over $25 Million A Year
Business ideas written for video x7pGM5FSTpI:
---
Processing video link: https://www.youtube.com/watch?v=LbWVysxsOhQ
Video Title: How Insomnia Cookies Went From College Side Hustle To $200 Million Company
Business ideas written for video LbWVysxsOhQ:
---
Processing video link: https://www.youtube.com/watch?v=zUN9KQpUgzM
Video Title: We Accidentally Started A Chocolate Company — Now It Brings In $162 Million A Year
Business ideas written for video zUN9KQpUgzM:
---
Processing video link: https://www.youtube.com/watch?v=PtLjacVrdFc
Video Title: How I Afford NYC Living On $17K A Year | Millennial Money
Business ideas written for video PtLjacVrdFc:
---
Processing video link: https://www.youtube.com/watch?v=7xbp7TYk7SU
Video Title: Bringing In $26.5 Million A Year From Our Photography Startup
Business ideas written for vide

ValueError: not enough values to unpack (expected 2, got 1)

In [68]:
from google.colab import files

# Make sure this filename matches the one you use in your script
files.download('cnbcmakeitexports1.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>